# IT492 Recommendation Systems
## Challenge 2
Group1: RecommendoCrew
1. Ayush Hirdani - 202101139
2. Sumukh Patel - 202101422
3. Takshay Makadia - 202101414
4. Akshat Prasad - 202101419
5. Arsh Jindal - 202103021

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

In [2]:
cols=['user_id', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'target']
usecol = ['user_id', 'song_id', 'target']

In [29]:
df = pd.read_csv("/kaggle/input/recommendation-system-challenge-music-suggestion/recommendation challenge (Music)/train.csv/train.csv", header=0, names=cols, usecols=usecol)
df

,user_id,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1
...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,1
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,0
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,1
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,1


In [4]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [5]:
data = df.groupby('user_id')['target'].count()

In [11]:
user_num = len(df['user_id'].unique())
song_num = len(df['song_id'].unique())

print(f'Number of Users: {user_num}')
print(f'Number of Songs: {song_num}')

Number of Users: 30755
Number of Songs: 359966


In [39]:
mapper = {}

label_cols = ['user_id', 'song_id']
for x in label_cols:
    mapper[x] = LabelEncoder()
    
for x in label_cols:
    df[x] = mapper[x].fit_transform(df[x])

In [42]:
df.head()

,user_id,song_id,target
0,8158,74679,1
1,17259,223479,1
2,17259,120758,1
3,17259,23707,1
4,8158,33308,1


In [43]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [60]:
class SongData(Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df[['user_id', 'song_id', 'target']]
        self.user = df['user_id']
        self.songs = df['song_id']
        self.target = df['target']
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self, idx):
        users = self.user[idx]
        songs = self.songs[idx]
        target = self.target[idx]
        
        return users, songs, target

In [67]:
class MatrixFactorization(nn.Module):
    def __init__(self, user_num, song_num, k):
        super().__init__()
        self.user_embed = nn.Embedding(num_embeddings=user_num, embedding_dim = k)
        self.song_embed = nn.Embedding(num_embeddings=song_num, embedding_dim = k)
    def forward(self, user, item):
        user_emb = self.user_embed(user)
        item_emb = self.song_embed(item)
        element_product = (user_emb*item_emb).sum(1)
        return element_product

In [80]:
model = MatrixFactorization(user_num, song_num, k=32)
model.to(device)
print(model)

MatrixFactorization(
  (user_embed): Embedding(30755, 32)
  (song_embed): Embedding(359966, 32)
)


In [71]:
train_ds = SongData(df)

In [ ]:
train_loader = DataLoader(train_ds, batch_size=2048, shuffle=True, num_workers=4)

In [78]:
opt = optim.Adam(model.parameters(), lr=0.001)

In [82]:
EPOCHS=4
loss_fn = nn.BCEWithLogitsLoss()
epoch_train_losses, epoch_val_losses = [], []

for epoch in range(EPOCHS):
    train_losses, val_losses = [], []
    model.train()
    
    for user, song, target in train_loader:
        user = user.to(device, dtype=torch.long)
        song = song.to(device, dtype=torch.long)
        target = target.to(device, dtype=torch.float)
        preds = model(user, song)
        loss = loss_fn(preds, target)
        train_losses.append(loss.item())
        opt.zero_grad()
        loss.backward()
        opt.step()
    epoch_train_loss = np.mean(train_losses)
    epoch_train_losses.append(epoch_train_loss)
    print(f'Epoch: {epoch}, Train Loss: {epoch_train_loss:0.1f}')

Epoch: 0, Train Loss: 2.4
Epoch: 1, Train Loss: 2.4
Epoch: 2, Train Loss: 2.4
Epoch: 3, Train Loss: 2.4


In [83]:
torch.save(model, 'model.pth')

In [87]:
test_cols=['id', 'msno', 'song_id']

In [89]:
test_df = pd.read_csv('/kaggle/input/recommendation-system-challenge-music-suggestion/recommendation challenge (Music)/test.csv/test.csv', usecols=test_cols)
test_df=test_df.rename(columns={'msno':'user_id'})
test_df

,id,user_id,song_id
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=
...,...,...,...
2556785,2556785,XmA/cIkXJ8jZUfsUc4bBfJVWMMqmylnPW0WVkg/iz0s=,wJoWzZd7AL+qX9xZWZwRTzGRYg0Lxcl5Pe+9n5hZgAQ=
2556786,2556786,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,KCJ8BlSfRQRgB7EVuzFvg52AhR8m2fT032MzN5ewbEI=
2556787,2556787,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,ySDsKJSnhFMESzC9mBhY0hnFEEBWDYx0FOMNcYDLb/A=
2556788,2556788,jvbujcxCExG0CrPShsEmZ6pePcHuRqru2OLEUw85iGk=,WXH8kL8e+0H2jY+s2Y6FpHyyHdaMYV3b6yfDa3FAt9U=


In [ ]:
for x in label_cols:
    test_df[x] = test_df[x].map(lambda s: '<unknown>' if s not in mapper[x].classes_ else s)
    mapper[x].classes = np.append(mapper[x].classes_, '<unknown>')
    test_df[x] = mapper[x].transform(test_df[x])

In [ ]:
test_df

In [ ]:
test_data_pos = {}

for index, row in test_df.iterrows():
    u = int(row['user_id'])
    i = int(row['song_id'])
    if u in test_data_pos:
        test_data_pos[u].append(i)
    else:
        test_data_pos[u] = [i]
    
test_data_pos